In [22]:
from config.loader import load_config
import numpy as np

# 加载配置
config = load_config()

# 访问配置项
db_host = config['robot_ip']
fx = config['DEPTH_INTR']['fx']
rotation_matrix = config['rotation_matrix']
translation_vector = config['translation_vector']

print(f"robot ip is: {db_host}")
print('try to find type and value of fx', '\n', type(fx), fx)
print('try to find type and value of translation_vector', '\n', type(translation_vector), translation_vector, type(translation_vector[1]), translation_vector[1])
print(len(translation_vector))
print(rotation_matrix)
a = np.eye(4)
a[:3, 3] = translation_vector
a[:3, :3] = rotation_matrix
a

robot ip is: 192.168.110.118
try to find type and value of fx 
 <class 'float'> 382.489
try to find type and value of translation_vector 
 <class 'list'> [-0.09674632, 0.03282877, 0.0294021] <class 'float'> 0.03282877
3
[[-0.00310044, 0.99999236, 0.00238261], [-0.99997374, -0.00311597, 0.00654238], [0.00654975, -0.00236226, 0.99997576]]


array([[-0.00310044,  0.99999236,  0.00238261, -0.09674632],
       [-0.99997374, -0.00311597,  0.00654238,  0.03282877],
       [ 0.00654975, -0.00236226,  0.99997576,  0.0294021 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [32]:
from Robotic_Arm.rm_robot_interface import *

# 实例化RoboticArm类
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)

# 创建机械臂连接，打印连接id
handle = arm.rm_create_robot_arm("192.168.110.118", 8080)
pose = [0.3, 0, 0.3, 3.14, 0, 0]
init = [0, 0, 0, 0, 0, 0]
params = rm_inverse_kinematics_params_t(init, pose, 1)
joint = arm.rm_algo_inverse_kinematics(params)
# print(type(joint), type(joint[0]), joint)
# a, b = arm.rm_get_current_arm_state()
# print(b)
# pose = b['pose']
# mat = arm.rm_algo_pos2matrix(pose)
# print(mat)
# out = arm.rm_algo_matrix2pos(mat)
# print(out, type(out))

print(arm.rm_movej(init, 20, 0, 0 ,1))
# print(arm.rm_movej_p(pose, 20, 0, 0, 1))
# print(arm.rm_movej(init, 20, 0, 0 ,1))
# print(arm.rm_movej(joint[1], 20, 0, 0 ,1))
# print(arm.rm_movej(init, 20, 0, 0 ,1))

arm.rm_delete_robot_arm()

current c api version:  1.0.6


0


0

In [4]:
from Robotic_Arm.rm_robot_interface import *
import numpy as np

arm_model = rm_robot_arm_model_e.RM_MODEL_RM_65_E  # RM_65机械臂
force_type = rm_force_type_e.RM_MODEL_RM_B_E  # 标准版
# 初始化算法的机械臂及末端型号
algo_handle = Algo(arm_model, force_type)

# 将位姿转为旋转矩阵
mat = algo_handle.rm_algo_pos2matrix([-0.177347, 0.438112, -0.215102, 2.09078, 0.942362, 2.39144])

arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
mat1 = arm.rm_algo_pos2matrix([-0.177347, 0.438112, -0.215102, 2.09078, 0.942362, 2.39144])
print(mat==mat1)
a = np.array(mat.data)
b = np.array(mat1.data)
print(a==b)


current c api version:  1.0.6
False
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [18]:
import numpy as np
from Robotic_Arm.rm_robot_interface import *

global arm
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
handle = arm.rm_create_robot_arm("192.168.110.118", 8080)
translation_vector = np.array([-0.09674632, 0.03282877, 0.0294021]) 
rotation_matrix = np.array([[-0.00310044 , 0.99999236 , 0.00238261],
                  [-0.99997374 ,-0.00311597  ,0.00654238],
                  [ 0.00654975 ,-0.00236226 , 0.99997576]])
h_eye = np.array([[-0.00310044 , 0.99999236 , 0.00238261, -0.09674632],
                  [-0.99997374 ,-0.00311597  ,0.00654238, 0.03282877],
                  [ 0.00654975 ,-0.00236226 , 0.99997576, 0.0294021],
                  [0, 0, 0, 1]])

_, dic1 = arm.rm_get_current_arm_state()    
current_joint, current_pose = dic1['joint'], dic1['pose']
matrix = arm.rm_algo_pos2matrix(current_pose)
h_current = np.array(matrix.data, dtype=np.float32).reshape(4, 4)
r_current = h_current[:3, :3]
pose_cam = h_current @ h_eye
print('the honogeneous matrix of robot end: ', '\n', h_current)
print('the honogeneous matrix of camera: ', '\n', pose_cam)
judgement = pose_cam[2, -1] - h_current[2, -1]
print('the difference of height between camera and robot end: ', '\n', judgement)
if judgement >= 0:
    print('相机没有朝下，不用管它')
else:
    print('相机朝下，需要旋转，请稍等')
    current_joint[-1] += 180
    print(arm.rm_movej(current_joint, 20, 0, 0, 1))

arm.rm_delete_robot_arm()

current c api version:  1.0.6
the honogeneous matrix of robot end:  
 [[ 9.4174999e-01 -0.0000000e+00 -3.3631384e-01 -2.0187500e-01]
 [ 0.0000000e+00  1.0000000e+00 -0.0000000e+00  7.0000001e-06]
 [ 3.3631384e-01  0.0000000e+00  9.4174999e-01  5.9295797e-01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
the honogeneous matrix of camera:  
 [[-0.00512261  0.94253726 -0.33406187 -0.30287418]
 [-0.99997374 -0.00311597  0.00654238  0.03283577]
 [ 0.00512551  0.33408662  0.94252847  0.58811027]
 [ 0.          0.          0.          1.        ]]
the difference of height between camera and robot end:  
 -0.0048476993650734546
相机朝下，需要旋转，请稍等
0


0

In [36]:
import pyrealsense2 as rs
import numpy as np
import cv2
from func01 import pixel_to_camera_coord
from Robotic_Arm.rm_robot_interface import *

# 实例化RoboticArm类
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)

# 创建机械臂连接，打印连接id
handle = arm.rm_create_robot_arm("192.168.110.118", 8080)
# print(handle.id)
pose_collect = []

# 初始化管道
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)  # RGB流
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)  # 深度流

# config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 6)  # RGB流
# config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 6)  # 深度流

pipeline.start(config)
align = rs.align(rs.stream.color)

# 鼠标回调函数
depth_scale = None
point_selected = False

def mouse_callback(event, x, y, flags, param):
    global depth_scale, point_selected, arm, pose_collect
    if event == cv2.EVENT_LBUTTONDOWN:
        # 获取深度帧
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        depth_frame = aligned_frames.get_depth_frame()
        
        # 获取深度比例因子
        if depth_scale is None:
            depth_sensor = pipeline.get_active_profile().get_device().first_depth_sensor()
            depth_scale = depth_sensor.get_depth_scale()
        
        # 获取深度值（米）
        depth_m = depth_frame.get_distance(x, y)
        # print('depth scale', depth_scale)
        # depth_m = depth_frame.get_distance(x, y) * depth_scale
        print(f"深度值（米）: {depth_m:.3f}")
        
        # 获取RGB内参
        color_profile = pipeline.get_active_profile().get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()
        fx = color_profile.fx
        fy = color_profile.fy
        cx = color_profile.ppx
        cy = color_profile.ppy
        # 坐标转换, 相机坐标系坐标
        Xc = (x - cx) / fx * depth_m
        Yc = (y - cy) / fy * depth_m
        Zc = depth_m
        point_cam = np.array([Xc, Yc, Zc])


        # 转换成机械臂基坐标系下的坐标
        R_cam_base = np.array(  [[-0.00310044 , 0.99999236 , 0.00238261],
                                [ -0.99997374 ,-0.00311597  ,0.00654238],
                                [ 0.00654975 ,-0.00236226 , 0.99997576]])
        t_cam_base = np.array( [[-0.09674632,0.03282877,0.0294021 ]])
        point_base = R_cam_base @ point_cam + t_cam_base
        point_base = np.array(point_base, dtype=np.float32).reshape(3, 1)
        # print(type(point_base), np.shape(point_base))
        a, b = arm.rm_get_current_arm_state()
        pose = b['pose']
        print(pose)
        mat = arm.rm_algo_pos2matrix(pose)
        # print(mat)
        mat = np.array(mat.data, dtype=np.float32).reshape(4, 4)
        rot, vec = mat[:3, :3], mat[:3, -1].reshape(1, 3)
        # print(type(rot), np.shape(rot), type(vec), np.shape(vec))

        point_base = rot.dot(point_base)
        point_base = point_base.reshape(1, 3)
        point_base += vec

        pose_collect.append(point_base)
        
        print(f"像素坐标： ({x}, {y})")
        print(f"相机坐标系坐标: ({Xc:.3f}, {Yc:.3f}, {Zc:.3f})")
        print(f'机械臂基坐标系下坐标： ({point_base})')
        point_selected = True

# 创建窗口并绑定回调
cv2.namedWindow('RGB & Depth')
cv2.setMouseCallback('RGB & Depth', mouse_callback)

try:
    while True:
        # 获取对齐后的帧
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.get_color_frame()
        depth_frame = aligned_frames.get_depth_frame()
        
        # 转换为numpy数组
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())
        # print('color_image', np.shape(color_image), 'depth_image', np.shape(depth_image))
        
        # 可视化深度图（彩色映射）
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        
        # 合并显示RGB和彩色深度图
        # combined_image = np.hstack((color_image, depth_image))
        combined_image = np.hstack((color_image, depth_colormap))
        
        # 显示图像
        cv2.imshow('RGB & Depth', combined_image)
        # cv2.imshow('rbg', color_image)
        # cv2.imshow('Depth', depth_image)
        
        if point_selected:
            # 在图像上标记选中的点
            # cv2.circle(color_image, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow('RGB & Depth', combined_image)
            point_selected = False
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            arm.rm_delete_robot_arm()
            break
finally:
    pipeline.stop()
    cv2.destroyAllWindows()

current c api version:  1.0.6
深度值（米）: 0.527
[-0.20807, 0.000295, 0.488525, 3.142, 0.326, 0.001]
像素坐标： (480, 261)
相机坐标系坐标: (0.131, 0.020, 0.527)
机械臂基坐标系下坐标： ([[-0.45803362  0.09516113 -0.01514164]])


In [35]:
a = [-0.43634102 , 0.04662181 ,-0.00998363]
b = [-0.43667188 ,-0.00388653 ,-0.00951245]

out = ((a[0] - b[0])**2 + (a[1] - b[1])**2 + (a[2] - b[2])**2)**0.5
print(out)

0.050511621321113816


In [44]:
from Robotic_Arm.rm_robot_interface import *

# 实例化RoboticArm类
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
init = [-0.20807, 0.000295, 0.488525, 3.142, 0.326, 0.001]
pose1 = [-0.5885288119316101, -0.23499751091003418, 0.14162935316562653, 3.15, 0.287, -0.045]

# 创建机械臂连接，打印连接id
handle = arm.rm_create_robot_arm("192.168.110.118", 8080)

a, b = arm.rm_get_current_arm_state()
print(b)
pose = b['pose']
print(pose)

print(arm.rm_movej_p(init, 20, 0, 0, 1))
print(arm.rm_movej_p(pose1, 10, 0, 0, 1))
print(arm.rm_movej_p(init, 20, 0, 0, 1))

arm.rm_delete_robot_arm()

current c api version:  1.0.6
{'joint': [-0.10400000214576721, 0.968999981880188, 45.44900131225586, 0.02800000086426735, 114.90299987792969, -0.1379999965429306], 'pose': [-0.208079, 0.000305, 0.488523, 3.142, 0.325, 0.0], 'err': {'err_len': 1, 'err': ['0x0000']}}
[-0.208079, 0.000305, 0.488523, 3.142, 0.325, 0.0]
0
1


Move command result is: false, current Device: 0, trajectory_connect: 0


0


0

In [1]:
import pyrealsense2 as rs
import numpy as np

# 初始化管道
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
# config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 6)
# config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 6)
pipeline.start(config)

# 获取 RGB 相机内参
color_frame = pipeline.wait_for_frames().get_color_frame()
color_intrin = color_frame.profile.as_video_stream_profile().intrinsics
fx, fy, cx, cy = color_intrin.fx, color_intrin.fy, color_intrin.ppx, color_intrin.ppy
dist_coeffs = color_intrin.coeffs  # 畸变系数
dist_coeffs = np.array(dist_coeffs)

# 获取深度相机内参（与 RGB 共享）
# depth_intrin = color_intrin  # D435i 的 RGB 和深度相机内参对齐

depth_frame = pipeline.wait_for_frames().get_depth_frame()
depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
fx1, fy1, cx1, cy1 = depth_intrin.fx, depth_intrin.fy, depth_intrin.ppx, depth_intrin.ppy
dist_coeffs1 = depth_intrin.coeffs  # 畸变系数
dist_coeffs1 = np.array(dist_coeffs1)

# 停止管道
pipeline.stop()

K = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]
], dtype=np.float32)

print(color_intrin, '\n', fx, fy, cx, cy, dist_coeffs, '\n', K)
print(depth_intrin, '\n', fx1, fy1, cx1, cy1, dist_coeffs1)

[ 640x480  p[328.541 237.377]  f[608.816 607.547]  Inverse Brown Conrady [0 0 0 0 0] ] 
 608.8164672851562 607.5466918945312 328.5406799316406 237.3766632080078 [0. 0. 0. 0. 0.] 
 [[608.81647   0.      328.54068]
 [  0.      607.5467  237.37666]
 [  0.        0.        1.     ]]
[ 640x480  p[318.509 239.128]  f[382.489 382.489]  Brown Conrady [0 0 0 0 0] ] 
 382.488525390625 382.488525390625 318.5091247558594 239.1276092529297 [0. 0. 0. 0. 0.]
